In [9]:
import numpy as np
from numba import njit
from numba import types
from numba import jitclass
from numba.typed import Dict, List

In [31]:
faf = {
    1: (0.75, 5000, 15), 
    2: (0.20, 3000, 20), 
    3: (0.05, 2000, 25),
}

In [32]:
# TODO use this instead of a mixted type dictionary for easier NUMBA processing
faf_action = {
    2025: [[1, 2, 3], [0.1, 0.65, 0.25]],
    2030: [[1, 2, 3], [0, 0.25, 0.75]],
}

In [3]:
# TODO build numba so EndUseReplacement is list of NUMBA
# conver Replacement year to be numba list
faf_action = {
    'EndUseReplacement': [[[1, 2, 3], [0.1, 0.65, 0.25]], [[1, 2, 3], [0, 0.25, 0.75]]],
    'ReplacementYear': [2025, 2035]
}

In [6]:
end_uses = {'faf': (faf, faf_action), 
            'faf2': (faf, faf_action), 
            'faf3': (faf, )}

In [7]:
building_obj = {
    'CustomerClass': 'Residential',
    'Segment': 'Single Family',
    'ConstructionVintage': 'Existing',
    'PrimaryFuelType': 'Electric',
    'EfficiencyLevel': 'Low',
    'Floorspace': 2000,
    'NumberBuildings': 1000,
    'StartYear': 2020,
    'EndYear': 2040,
    'EndUses': end_uses,
}

In [8]:
def eu_dict_to_numba(eu_dict):
    """Convert end use dictionary to numba dictionary"""
    eu_dict_n = Dict.empty(
        key_type = types.int64,
        value_type = types.float64[:],
    )
    for key, value in eu_dict.items():
        eu_dict_n[key] = np.asarray(value)
    return eu_dict_n

In [ ]:
def action_dict_to_numba(action_dict):
    """Convert action dictionary to numba dictionary"""
    action_dict_n = Dict.empyt(
        key_type = types.unicode_type,
        value_type = List()
    )
    
    for key, value in action_dict.items():
        if key == 'EndUseReplacement':
            eu_level = List()
            eu_sat = List()
            for n in value:
                eu_level.append(n[0])
                eu_sat.append(n[1])
        

In [ ]:
@njit
def create_stock_no_replacement(start, end, end_use, n):
    """If there is no action then stock will stay constant"""
    stock = np.ones((len(end_use), end - start + 1))
    # will only create a 1d array
    stock_0 = np.asarray([x[0] * n for x in end_use.values()])
    # need to reshape stock_0 to (n, 1) array for broadcasting
    return stock * stock_0.reshape((stock_0.size, 1))

In [ ]:
def create_stock_replacement_prob(start, end, n, end_use, action):
    """stock replacement probability matrix"""
    # set init vars
    eu_base = min(end_use.keys())
    eu_repl = action['EndUseReplacement']
    repl_year = action['ReplacementYear']
    
    # build empty replacement matrix (# of equipment efficiency levels, n_periods)
    repl_arr = np.ones((len(eu_obj), end - start))
    prob_0 = np.asarray([x[0] for x in end_use_obj.values()])
    repl_arr = repl_arr * prob_0[:, None]

    # loop over replacement conditions
    # outerloop for each year with efficiecy ramp
    for i in zip(eu_repl, repl_year):
        # inner loop for populating replacement matrix for efficiency ramp condition
        for x in zip(i[0][0], i[0][1]):
            repl_arr[x[0] - eu_base, (i[1] - start - 1):] = x[1]
                       
    return repl_arr

In [ ]:
@njit
def stock_turnover_calc(stock_mat, eul_mat, stock_repl_mat):
    """Do the numerical work of turnover calc"""
    t_mat = np.full_like(stock_repl_mat, 0)
    for i in np.ndindex(t_mat.shape[1]):
        st_i = stock_mat[:, i[0]] / eul_mat[:, i[0]]
        t_mat[:, i[0]] = st_i.sum() * stock_repl_mat[:, i[0]]
        stock_mat[:, i[0] + 1] = (stock_mat[:, i[0]] - st_i + t_mat[:, i[0]]).ravel()
    return stock_mat

In [ ]:
def create_stock_turnover(start, end, action_object, n_buildings):
    """Container to handle turnover calculation"""
    # set init vars
    eu_obj = action_object['EndUseObject']
    repl =  action_object['EndUseReplacement']
    
    # get initial stock values
    stock_0 = np.asarray([x[0] * n_buildings for x in eu_obj.values()])
    uec_0 =  np.asarray([x[1] for x in eu_obj.values()])
    eul_0 = np.asarray([x[2] for x in eu_obj.values()])
    
    # create empty stock array
    stock_mat = np.zeros((stock_0.shape[0], end - start + 1))
    uec_mat = np.ones((uec_0.shape[0], end - start + 1)) * uec_0[:, None] 
    eul_mat = np.ones((eul_0.shape[0], end - start + 1)) * eul_0[:, None] 
    
    # populate initial values
    stock_mat[:, 0] = stock_0
    
    # replacement array
    stock_repl_mat = create_stock_ramp_prob_mat(action_object)
    return stock_turnover_calc(stock_mat, eul_mat, stock_repl_mat)

In [ ]:
def create_building_mat(building_obj):
    """Initiate builing level calcuation"""
    # extract variables for calcuation
    start = building_obj['StartYear']
    end = building_obj['EndYear']
    floorspace = building_obj['Floorspace']
    n = building_obj['NumberBuildings']
    end_uses = building_obj['EndUses']
    # loop through all end-uses
    end_use_lst = []
    # loop over each end use
    for value in end_uses.values():
        # create numba dict
        # if there is only 1 item then no action
        eu_dict_n  = eu_dict_to_numba(value[0])
        if len(value) == 1:
            end_use_lst.append(create_stock_no_replacement(start, end, eu_dict_n, n))
        else:
            create_stock_turnover()
    return end_use_lst